In [32]:
# import
import pandas as pd
import numpy as np

In [36]:
# load data
train = pd.read_csv('../data/train.csv')

# show the first and last times in the test set pickup_datetime
# concert the pickup_datetime column to a datetime object
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'])
train["dropoff_datetime"] = pd.to_datetime(train["dropoff_datetime"])

print("Date Range:")
print(train['pickup_datetime'].min())
print(train['pickup_datetime'].max())

print("\nGrid Range:")
min_lat = min(train["pickup_latitude"].min(), train['dropoff_latitude'].min())
min_long = min(train["pickup_longitude"].min(), train['dropoff_longitude'].min())
max_lat = max(train["pickup_latitude"].max(), train['dropoff_latitude'].max())
max_long = max(train["pickup_longitude"].max(), train['dropoff_longitude'].max())
print(min_lat, min_long)
print(max_lat, max_long)

# drop extra columns
train = train.drop(['id', 'vendor_id', "store_and_fwd_flag"], axis=1)

# apply the floor("H") operation to every row in test["hour"]
# train['hour'] = train['pickup_datetime'].apply(lambda date: date.floor('H'))
train['hour'] = train['pickup_datetime'].dt.floor('H')

Date Range:
2016-01-01 00:00:17
2016-06-30 23:59:39

Grid Range:
32.1811408996582 -121.93334197998048
51.88108444213867 -61.33552932739258


In [37]:
train.head()

,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,hour
0,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,455,2016-03-14 17:00:00
1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,663,2016-06-12 00:00:00
2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,2124,2016-01-19 11:00:00
3,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,429,2016-04-06 19:00:00
4,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,435,2016-03-26 13:00:00


In [38]:
# load weather data
# https://www.kaggle.com/datasets/aadimator/nyc-weather-2016-to-2022
weather = pd.read_csv('../data/weather.csv')
weather = weather.dropna()

# drop all rows where the time is after 2016-07-01
weather['time'] = pd.to_datetime(weather['time'])
weather = weather[weather['time'] <= '2016-07-01']

# drop extraneous columns and show
weather = weather.drop(columns=['rain (mm)', 'cloudcover_low (%)', 'cloudcover_mid (%)', 'cloudcover_high (%)', 'windspeed_10m (km/h)', 'winddirection_10m (°)'])
print(weather['time'].min())
print(weather['time'].max())

2016-01-01 00:00:00
2016-07-01 00:00:00


In [39]:
# left join test and weather on hour == time
taxis = train.merge(weather, left_on='hour', right_on='time', how='left')
taxis = taxis.drop(columns=['time', 'hour'])

In [40]:
taxis.head()

,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,temperature_2m (°C),precipitation (mm),cloudcover (%)
0,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,455,6.4,0.2,100.0
1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,663,28.8,0.0,5.0
2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,2124,-7.6,0.0,0.0
3,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,429,7.9,0.0,2.0
4,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,435,4.7,0.0,5.0


In [41]:
taxis.describe()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,temperature_2m (°C),precipitation (mm),cloudcover (%)
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02,1.029680e+01,9.552324e-02,4.417891e+01
std,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03,9.031819e+00,4.173544e-01,3.770505e+01
min,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00,-1.680000e+01,0.000000e+00,0.000000e+00
25%,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02,3.200000e+00,0.000000e+00,7.000000e+00
50%,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02,1.030000e+01,0.000000e+00,3.300000e+01
75%,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03,1.710000e+01,0.000000e+00,8.500000e+01
max,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06,3.070000e+01,7.900000e+00,1.000000e+02


In [42]:
taxis.dtypes

pickup_datetime        datetime64[ns]
dropoff_datetime       datetime64[ns]
passenger_count                 int64
pickup_longitude              float64
pickup_latitude               float64
dropoff_longitude             float64
dropoff_latitude              float64
trip_duration                   int64
temperature_2m (°C)           float64
precipitation (mm)            float64
cloudcover (%)                float64
dtype: object